In [ ]:
from langdetect import detect

In [1]:
import pandas as pd
import seaborn as sns
import string 
import matplotlib.pyplot as plt
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import warnings

warnings.filterwarnings("ignore")
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bertille.lekeufack\AppData\Roaming\nltk_data.
[nltk_data]     ..
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bertille.lekeufack\AppData\Roaming\nltk_data.
[nltk_data]     ..
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
#df=pd.read_excel('final_210416.xlsx',engine='openpyxl')
df=pd.read_excel('message_cord1.xlsx',engine='openpyxl')
#msg=msg.head(190)

In [ ]:
df['Messages'] = df['Messages'].astype(str)

In [ ]:
msg= df.head(190)
msg

In [ ]:
texl70 = msg['text']
langdet = []
    
for i in range(len(msg)):                                         
    try:                                                          
        lang=detect(texl70[i])                                      
    except:                                                       
        lang=''                                                  
        print("This row throws error:", texl70[i])                 
    langdet.append(lang)                

In [ ]:
msg['lang']=langdet
msg.head()

In [ ]:
msg=msg.loc[msg['lang']=='fr']
msg.shape

In [ ]:
msg['catégorie'] = msg['catégorie'].replace({ 'contrat' : 0,'demande de renseignement' : 1,'interne Axxès' : 2 ,'facture' : 3, 'commande de badge' : 4, 'réclamation qualité de service' : 5,'réclamation facture':6,'remplacement badge':7,'réclamation logistique':8,
                                            'amende Belgique':9,'stef':10,'no message':11})


In [ ]:
msg.head(30)

In [ ]:
msg = msg.drop(['Messages',  'occurence','lang'], axis=1)

In [ ]:
sns.countplot(msg.catégorie)

In [ ]:
msg.groupby('catégorie').describe()

In [ ]:
msg=msg.drop_duplicates()
msg.groupby('catégorie').describe()

In [ ]:
msg['Messages']=msg['Messages'].astype(str)

In [ ]:
msg.catégorie.value_counts()

In [ ]:
msg['length']=msg['Messages'].apply(len)
msg.head()

In [ ]:
msg[msg.catégorie==0].describe()

In [ ]:
sns.distplot(a=msg[msg['catégorie']==0].length,kde=False)

In [ ]:
msg[msg.catégorie==1].describe()

In [ ]:
sns.distplot(a=msg[msg['catégorie']==1].length,kde=False)#kernel density estimation

In [ ]:
tab=[]

for i in msg.index:
    
    msg['Messages'][i]=re.sub('([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)', '', str(msg['Messages'][i]))#email
    #phone
    msg['Messages'][i]=re.sub(r"/\d{10}|\+33\d{9}|\+33\s\d{1}\s\d{2}\s\d{2}\s\d{2}\s\d{2}|\d{2}\s\d{2}\s\d{2}\s\d{2}\s\d{2}/g",' ',str(msg['Messages'][i]))#phone francais
    msg['Messages'][i]=re.sub(r'(^(\+33 |0)[1-6]{1}(([0-9]{2}){4})|((\s[0-9]{2}){4})|((-[0-9]{2}){4})$)','',str(msg['Messages'][i]))
    msg['Messages'][i]=re.sub("((?:\+\d{2}[-\.\s]??|\d{4}[-\.\s]??)?(?:\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4}))",'', str(msg['Messages'][i]))#phone
    msg['Messages'][i]=re.sub('\(?\d{2}\)?[-.\s]?\d{2}[-.\s]?\d{2}[-.\s]?\d{2}[-.\s]?\d{2}','',str(msg['Messages'][i]))
    msg['Messages'][i]=re.sub('\(?\d{2}\)?[-.\s]?\d{2}[-.\s]?\d{2}[-.\s]?\d{2}[-.\s]','',str(msg['Messages'][i]))
    msg['Messages'][i]=re.sub('\(?\d{1}\)?[-.\s]?\d{2}[-.\s]?\d{2}[-.\s]?\d{2}[-.\s]','',str(msg['Messages'][i]))
    msg['Messages'][i]=re.sub('\(?\d{2}\)?[-.\s]?\d{5}[-.\s]?\d{3}','',str(msg['Messages'][i]))
    msg['Messages'][i]=re.sub('\(?\d{1}\)?[-.\s]?\d{2}[-.\s]?\d{2}[-.\s]?\d{3}','',str(msg['Messages'][i]))
    msg['Messages'][i]=re.sub('\(?\d{2}\)?[-.\s]?\d{3}[-.\s]?\d{3}[-.\s]?\d{3}','',str(msg['Messages'][i]))
    #Emoji
    emoji= re.compile(u'['u'\U0001F300-\U0001F64F'u'\U0001F680-\U0001F6FF'u'\u2600-\u26FF\u2700-\u27BF]+', re.UNICODE)
    msg['Messages'][i]=re.sub(emoji,' ',str(msg['Messages'][i]))
    #text
    msg['Messages'][i]= msg['Messages'][i].replace("Avant d\'imprimer cet e-mail, réfléchissez à l\'impact sur l\'environnementBefore you print this mail, think about the impact on the environment",'')
    dele = ["Mail:","Email:","e-mail","Email : ","mail","Tél : +","Fax : +","tél.: + ()","port.: + () ","fax.: + ()"]
    #df['Text'][i]= df['Text'][i].replace(dele,'')
    msg['Messages'][i]= msg['Messages'][i].replace("Mail:",'')
    msg['Messages'][i]= msg['Messages'][i].replace("Email:",'')
    msg['Messages'][i]= msg['Messages'][i].replace("E-mail",'')
    msg['Messages'][i]= msg['Messages'][i].replace("Email : ",'')
    msg['Messages'][i]= msg['Messages'][i].replace("mail",'')
    msg['Messages'][i]=re.sub(r'\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4}','',str(msg['Messages'][i]))
    #df['Text'][i]=re.sub(r'\d+','',str(df['Text'][i]))#Tél : +      Fax : +, tél.: + ()       fax.: + ()      port.: + () 
    msg['Messages'][i]= re.sub("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",'',msg['Messages'][i])#url
    msg['Messages'][i]= re.sub('(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})',' ',str(msg['Messages'][i]))#url www
    msg['Messages'][i]= re.sub('Ce message et toutes les Pieces jointes (ci apres : le message) sont etablis a l\'intention exclusive de ses destinataires et sont confidentiels.Toute utilisation de ce message non conforme a sa destination, toute diffusion ou toute publication totale ou partielle est interdite sauf autorisation prealable.Si vous recevez ce message par erreur, merci de le detruire et d\' avertir immediatement l\'expediteur.Les communications sur Internet n\'etant pas securisees, le Groupe APRR decline toute responsabilite quant au contenu de ce message.This message and all attached documents (hereafter "the message") are intended for the exclusive attention of its recipients and are confidential. Any use of this message, for which it was not intended, any distribution or any total or partial publication is prohibited unless previously approved.If you receive this message in error, please destroy it and immediately notify the sender thereof.Since communication by Internet is not secure, Group APRR cannot accept any responsibility regarding the contents of this message.',' ',str(msg['Messages'][i]))
    #df['Text'][i]= re.sub('SCAN <>Cet  a été envoyé par "copieurRICOH" (MP C3004ex).Date de numérisation: 07.08.2020 17:17:25 (+0200)Contacter si besoin :  ','',str(df['Text'][i]))
    msg['Messages'][i]= re.sub('Ce message est généré automatiquement. Merci de ne pas y répondre.',' ',str(msg['Messages'][i]))
    msg['Messages'][i]= re.sub('Ce message et toutes les Pieces jointes (ci apres : le message) sont etablis a l\'intention exclusive de ses destinataires et sont confidentiels.Toute utilisation de ce message non conforme a sa destination, toute diffusion ou toute publication totale ou partielle est interdite sauf autorisation prealable.Si vous recevez ce message par erreur, merci de le detruire et d\' avertir immediatement l\'expediteur.Les communications sur Internet n\'etant pas securisees, le Groupe APRR decline toute responsabilite quant au contenu de ce message.This message and all attached documents (hereafter "the message") are intended for the exclusive attention of its recipients and are confidential. Any use of this message, for which it was not intended, any distribution or any total or partial publication is prohibited unless previously approved.If you receive this message in error, please destroy it and immediately notify the sender thereof.Since communication by Internet is not secure, Group APRR cannot accept any responsibility regarding the contents of this message.',' ',str(msg['Messages'][i]))
    msg['Messages'][i]= re.sub('This message and any attachments are confidential and intended for the named addressee(s) only.If you have received this message in error, please notify immediately the sender, then deletethe message. Any unauthorized modification, edition, use or dissemination is prohibited.The sender shall not be liable for this message if it has been modified, altered, falsified, infectedby a virus or even edited or disseminated without authorization.',' ',str(msg['Messages'][i]))
    msg['Messages'][i]= re.sub('This  transmission (message and any attached files) may contain information that is proprietary, privileged and/or confidential to Veolia Environnement and/or its affiliates and is intended exclusively for the person(s) to whom it is addressed. If you are not the intended recipient, please notify the sender by return  and delete all copies of this , including all attachments. Unless expressly authorized, any use, disclosure, publication, retransmission or dissemination of this  and/or of its attachments is strictly prohibited. Ce message electronique et ses fichiers attaches sont strictement confidentiels et peuvent contenir des elements dont Veolia Environnement et/ou l\'une de ses entites affiliees sont proprietaires. Ils sont donc destines a l\'usage de leurs seuls destinataires. Si vous avez recu ce message par erreur, merci de le retourner a son emetteur et de le detruire ainsi que toutes les pieces attachees. L\'utilisation, la divulgation, la publication, la distribution, ou la reproduction non expressement autorisees de ce message et de ses pieces attachees sont interdites.',' ',str(msg['Messages'][i]))
    msg['Messages'][i]= re.sub('This  transmission (message and any attached files) may contain information that is proprietary, privileged and/or confidential to Veolia Environnement and/or its affiliates and is intended exclusively for the person(s) to whom it is addressed. If you are not the intended recipient, please notify the sender by return  and delete all copies of this , including all attachments. Unless expressly authorized, any use, disclosure, publication, retransmission or dissemination of this  and/or of its attachments is strictly prohibited.  Ce message electronique et ses fichiers attaches sont strictement confidentiels et peuvent contenir des elements dont Veolia Environnement et/ou l\'une de ses entites affiliees sont proprietaires. Ils sont donc destines a l\'usage de leurs seuls destinataires. Si vous avez recu ce message par erreur, merci de le retourner a son emetteur et de le detruire ainsi que toutes les pieces attachees. L\'utilisation, la divulgation, la publication, la distribution, ou la reproduction non expressement autorisees de ce message et de ses pieces attachees sont interdites.',' ',str(msg['Messages'][i]))
    msg['Messages'][i]= re.sub('Le contenu de ce message est strictement réservé à l\'usage exclusif de son destinataire et contient de l\'information privilégiée et confidentielle. Toute lecture, divulgation, distribution ou copie non autorisée de cette communication est formellement interdite. Si vous avez reçu cette communication par erreur, veuillez la détruire immédiatement.','',str(msg['Messages'][i]))
    msg['Messages'][i]= re.sub('Ce message et toutes les Pieces jointes (ci apres : le message) sont etablis a l\'intention exclusive de ses destinataires et sont confidentiels.Toute utilisation de ce message non conforme a sa destination, toute diffusion ou toute publication totale ou partielle est interdite sauf autorisation prealable.Si vous recevez ce message par erreur, merci de le detruire et d\' avertir immediatement l\'expediteur.Les communications sur Internet n\'etant pas securisees, le Groupe APRR decline toute responsabilite quant au contenu de ce message.This message and all attached documents (hereafter "the message") are intended for the exclusive attention of its recipients and are confidential. Any use of this message, for which it was not intended, any distribution or any total or partial publication is prohibited unless previously approved.If you receive this message in error, please destroy it and immediately notify the sender thereof.Since communication by Internet is not secure, Group APRR cannot accept any responsibility regarding the contents of this message.',' ',str(msg['Messages'][i]))
    msg['Messages'][i]= re.sub('Ce message et les pièces jointes sont confidentiels et établis à l\'attention exclusive de ses destinataires. Toute utilisation ou diffusion, même partielle, non autorisée est interdite. Tout message électronique est susceptible d\'altération. Brink\'s décline toute responsabilité au titre de ce message s\'il a été altéré, déformé ou falsifié. Si vous n\'êtes pas le destinataire de ce message, merci de le détruire et d\'avertir l\'expéditeur. This message and any attachments are confidential and intended solely for the addressees. Any unauthorized use or disclosure, either whole or partial is prohibited. E-s are susceptible to alteration. Brink\'s shall not be liable for the message if altered, changed or falsified. If you are not the intended recipient of this message, please delete it and notify the sender.',' ',str(msg['Messages'][i]))
    msg['Messages'][i]= re.sub('Ce message et toutes les Pieces jointes (ci apres : le message) sont etablis a l\'intention exclusive de ses destinataires et sont confidentiels.Toute utilisation de ce message non conforme a sa destination, toute diffusion ou toute publication totale ou partielle est interdite sauf autorisation prealable.Si vous recevez ce message par erreur, merci de le detruire et d\' avertir immediatement l\'expediteur.Les communications sur Internet n\'etant pas securisees, le Groupe APRR decline toute responsabilite quant au contenu de ce message.','',str(msg['Messages'][i]))
    
    print(msg['Messages'][i])

In [ ]:
t = []
t = msg
dfprint = pd.DataFrame(t)
dfprint.to_excel("message_regex_messages.xlsx", index = False)

In [24]:
df=pd.read_excel('message_cord1.xlsx',engine='openpyxl')
msg=df.head(250)

In [25]:
msg = msg.drop(['Messages',  'occurence','lang'], axis=1)

In [26]:
msg['catégorie'] = msg['catégorie'].replace({ 'contrat' : 0,'demande de renseignement' : 1,'interne Axxès' : 2 ,'facture' : 3, 'commande de badge' : 4, 'réclamation qualité de service' : 5,'réclamation facture':6,'remplacement badge':7,'réclamation logistique':8,
                                            'amende Belgique':9,'stef':10,'no message':11})


In [27]:
msg['cleaned_text']=""

for i in msg.index:
    
    updated_list=[]
    for j in range(len(msg['text'][i])):
        if msg['text'][i][j] not in string.punctuation:
            if msg['text'][i][j].isdigit()==False:
                updated_list.append(msg['text'][i][j])
    updated_string="".join(updated_list)
    msg['cleaned_text'][i]=updated_string

msg.drop(['text'],axis=1,inplace=True)
msg.head() 

,catégorie,cleaned_text
0,3,BonjourCijoint le mandat sepa et le rib
1,4,DANIEL GUEGANSOCIETE DES TRANSPORTS GUEGANCl...
2,4,Bonjour J’ai essayé de faire la modification ...
3,4,Bonjour merci de me faire un badge avec la géo...
4,5,Bonjour Lappareil XXXX a été attribué au camio...


In [28]:
msg['token']=""

for i in msg.index:
    
    msg['token'][i]=re.split("\W+",msg['cleaned_text'][i].lower())

msg.head()

,catégorie,cleaned_text,token
0,3,BonjourCijoint le mandat sepa et le rib,"[bonjourcijoint, le, mandat, sepa, et, le, rib]"
1,4,DANIEL GUEGANSOCIETE DES TRANSPORTS GUEGANCl...,"[daniel, guegansociete, des, transports, guega..."
2,4,Bonjour J’ai essayé de faire la modification ...,"[, bonjour, j, ai, essayé, de, faire, la, modi..."
3,4,Bonjour merci de me faire un badge avec la géo...,"[bonjour, merci, de, me, faire, un, badge, ave..."
4,5,Bonjour Lappareil XXXX a été attribué au camio...,"[bonjour, lappareil, xxxx, a, été, attribué, a..."


In [29]:
msg['updated_token']=""
stopwords=nltk.corpus.stopwords.words('french')

for i in msg.index:
    updated_list=[]
    for j in range(len(msg['token'][i])):
        if msg['token'][i][j] not in stopwords:
            updated_list.append(msg['token'][i][j])
    msg['updated_token'][i]=updated_list

msg.drop(['token'],axis=1,inplace=True)
msg.head()

,catégorie,cleaned_text,updated_token
0,3,BonjourCijoint le mandat sepa et le rib,"[bonjourcijoint, mandat, sepa, rib]"
1,4,DANIEL GUEGANSOCIETE DES TRANSPORTS GUEGANCl...,"[daniel, guegansociete, transports, gueganclie..."
2,4,Bonjour J’ai essayé de faire la modification ...,"[, bonjour, essayé, faire, modification, badge..."
3,4,Bonjour merci de me faire un badge avec la géo...,"[bonjour, merci, faire, badge, géolocalisation..."
4,5,Bonjour Lappareil XXXX a été attribué au camio...,"[bonjour, lappareil, xxxx, a, attribué, camion..."


In [30]:
msg['lem_text']=""
wordlem=nltk.WordNetLemmatizer()

for i in msg.index:
    updated_list=[]
    for j in range(len(msg['updated_token'][i])):
        updated_list.append(wordlem.lemmatize(msg['updated_token'][i][j]))
    msg['lem_text'][i]=updated_list 

msg.drop(['updated_token'],axis=1,inplace=True)
#msg.drop(['lang'],axis=1,inplace=True)
msg.head()

,catégorie,cleaned_text,lem_text
0,3,BonjourCijoint le mandat sepa et le rib,"[bonjourcijoint, mandat, sepa, rib]"
1,4,DANIEL GUEGANSOCIETE DES TRANSPORTS GUEGANCl...,"[daniel, guegansociete, transport, gueganclien..."
2,4,Bonjour J’ai essayé de faire la modification ...,"[, bonjour, essayé, faire, modification, badge..."
3,4,Bonjour merci de me faire un badge avec la géo...,"[bonjour, merci, faire, badge, géolocalisation..."
4,5,Bonjour Lappareil XXXX a été attribué au camio...,"[bonjour, lappareil, xxxx, a, attribué, camion..."


In [31]:
#data cleaning/preprocessing - merging token
msg['final_text']=""

for i in msg.index:
    updated_string=" ".join(msg['lem_text'][i])
    msg['final_text'][i]=updated_string

msg.drop(['cleaned_text','lem_text'],axis=1,inplace=True)
msg.head()

,catégorie,final_text
0,3,bonjourcijoint mandat sepa rib
1,4,daniel guegansociete transport gueganclient xx...
2,4,bonjour essayé faire modification badge ec k ...
3,4,bonjour merci faire badge géolocalisation inté...
4,5,bonjour lappareil xxxx a attribué camion za na...


In [32]:
y=pd.DataFrame(msg.catégorie)
x=msg.drop(['catégorie'],axis=1)

In [33]:
#splitting the data (80:20 ratio)
x_train,x_val,y_train,y_val=train_test_split(x,y,train_size=0.8,test_size=0.2,random_state=0)

In [34]:
#count vectorization 
cv=CountVectorizer(max_features=5000)
temp_train=cv.fit_transform(x_train['final_text']).toarray()
temp_val=cv.transform(x_val['final_text']).toarray()

In [35]:
#tfidf
tf=TfidfTransformer()
temp_train=tf.fit_transform(temp_train)
temp_val=tf.transform(temp_val)

In [36]:
#ajout des valeurs tampon au dataset
temp_train=pd.DataFrame(temp_train.toarray(),index=x_train.index)
temp_val=pd.DataFrame(temp_val.toarray(),index=x_val.index)
x_train=pd.concat([x_train,temp_train],axis=1,sort=False)
x_val=pd.concat([x_val,temp_val],axis=1,sort=False)

x_train.head()

,final_text,0,1,2,3,4,5,6,7,8,...,2082,2083,2084,2085,2086,2087,2088,2089,2090,2091
71,bonjourveuillez trouver cijoint bon commande ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
160,bonjour reviens vers concernant livraison télé...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.104982,0.0,0.0,0.0,0.0,0.0,0.000000
180,bonjour floriane suite discussion karima perme...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.155565
75,madame monsieur prie trouver cijoint confirmat...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
59,bonjour entreprise a changé compte bancaire de...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000


In [37]:
#Suppression de la colonne final _text
x_train.drop(['final_text'],axis=1,inplace=True)
x_val.drop(['final_text'],axis=1,inplace=True)

x_train.head()

,0,1,2,3,4,5,6,7,8,9,...,2082,2083,2084,2085,2086,2087,2088,2089,2090,2091
71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
160,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.104982,0.0,0.0,0.0,0.0,0.0,0.000000
180,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.155565
75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000


In [38]:
#msg['catégorie']=msg['catégorie'].astype(int)

In [39]:
#convertion du label en entier
y_train=y_train.astype(int)
y_val=y_val.astype(int)

In [51]:
#Multinomial Naive Bayes
model=MultinomialNB()
model.fit(x_train,y_train)
y_preds=model.predict(x_val)
print("Multinomial Naive Bayes:",accuracy_score(y_val,y_preds))
from sklearn.metrics import f1_score
f1_score(y_val, y_preds, average='macro')

f1_score(y_val, y_preds, average='micro')

f1_score(y_val, y_preds, average='weighted')

Multinomial Naive Bayes: 0.42


0.24845070422535212

In [41]:
from sklearn.linear_model import SGDClassifier
model=SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)
model.fit(x_train,y_train)
y_preds=model.predict(x_val)
print("SGD:",accuracy_score(y_val,y_preds))

SGD: 0.66


In [42]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(n_jobs=1, C=1e5)
model.fit(x_train,y_train)
y_preds=model.predict(x_val)
print("Logistic regression:",accuracy_score(y_val,y_preds))


Logistic regression: 0.68


In [43]:
model=DecisionTreeClassifier(random_state=0)
model.fit(x_train,y_train)
y_preds=model.predict(x_val)
print("Decision Tree:",accuracy_score(y_val,y_preds))

Decision Tree: 0.5


In [22]:
#from sklearn.preprocessing import StandardScaler
#Scaler_X = StandardScaler()
#x_train = Scaler_X.fit_transform(x_train)
#x_val = Scaler_X.transform(x_val)

In [49]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from collections import Counter
smt=SMOTE(sampling_strategy = 'majority')
x_train_sm,y_train_sm=smt.fit_resample(x_train,y_train)

model=model.fit(x_train_sm,y_train_sm)
y_preds=model.predict(x_val)
print("Logistic regression:",accuracy_score(y_val,y_preds))

ValueError: 'sampling_strategy'='majority' cannot be used with over-sampler.

In [ ]:
#Random Forest
model=RandomForestClassifier(n_estimators=10,random_state=0,n_jobs=-1)
model.fit(x_train,y_train)
y_preds=model.predict(x_val)
print("Random Forest:",accuracy_score(y_val,y_preds))


In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score

In [ ]:
all_predictions = model.predict('je souhaite signer un contrat')


In [ ]:
all_predictions = model.predict(x_val)
pred = pd.DataFrame(data=all_predictions)

pred.head(30)

In [ ]:
sns.heatmap(confusion_matrix(y_val, y_preds), annot=True)

In [ ]:
plt.ylabel("True classes")
plt.xlabel("Predicted classes")
plt.show()

In [ ]:
print(confusion_matrix(y_val, y_preds))